In [ ]:
#NNet.py
import os
import sys
import time

import numpy as np
from tqdm import tqdm

sys.path.append('../../')
from utils import *
from NeuralNet import NeuralNet

import torch
import torch.optim as optim


args = dotdict({
    'lr': 0.001,
    'dropout': 0.3,
    'epochs': 10,
    'batch_size': 64,
    'cuda': torch.cuda.is_available(),
    'num_channels': 512,
})


class NNetWrapper(NeuralNet):
    def __init__(self, game):
        self.nnet = HexNNet(game, args)
        self.board_x, self.board_y = game.getBoardSize()
        self.action_size = game.getActionSize()

        if args.cuda:
            self.nnet.cuda()

    def train(self, examples):
        """
        examples: list of examples, each example is of form (board, pi, v)
        """
        optimizer = optim.Adam(self.nnet.parameters())

        for epoch in range(args.epochs):
            print('EPOCH ::: ' + str(epoch + 1))
            self.nnet.train()
            pi_losses = AverageMeter()
            v_losses = AverageMeter()

            batch_count = int(len(examples) / args.batch_size)

            t = tqdm(range(batch_count), desc='Training Net')
            for _ in t:
                sample_ids = np.random.randint(len(examples), size=args.batch_size)
                boards, pis, vs = list(zip(*[examples[i] for i in sample_ids]))
                boards = torch.FloatTensor(np.array(boards).astype(np.float64))
                target_pis = torch.FloatTensor(np.array(pis))
                target_vs = torch.FloatTensor(np.array(vs).astype(np.float64))

                # predict
                if args.cuda:
                    boards, target_pis, target_vs = boards.contiguous().cuda(), target_pis.contiguous().cuda(), target_vs.contiguous().cuda()

                # compute output
                out_pi, out_v = self.nnet(boards)
                l_pi = self.loss_pi(target_pis, out_pi)
                l_v = self.loss_v(target_vs, out_v)
                total_loss = l_pi + l_v

                # record loss
                pi_losses.update(l_pi.item(), boards.size(0))
                v_losses.update(l_v.item(), boards.size(0))
                t.set_postfix(Loss_pi=pi_losses, Loss_v=v_losses)

                # compute gradient and do SGD step
                optimizer.zero_grad()
                total_loss.backward()
                optimizer.step()

    def predict(self, board):
        """
        board: np array with board
        """
        # timing
        start = time.time()

        # preparing input
        board = torch.FloatTensor(board.astype(np.float64))
        if args.cuda: board = board.contiguous().cuda()
        board = board.view(1, self.board_x, self.board_y)
        self.nnet.eval()
        with torch.no_grad():
            pi, v = self.nnet(board)

        # print('PREDICTION TIME TAKEN : {0:03f}'.format(time.time()-start))
        return torch.exp(pi).data.cpu().numpy()[0], v.data.cpu().numpy()[0]

    def loss_pi(self, targets, outputs):
        return -torch.sum(targets * outputs) / targets.size()[0]

    def loss_v(self, targets, outputs):
        return torch.sum((targets - outputs.view(-1)) ** 2) / targets.size()[0]

    def save_checkpoint(self, folder='checkpoint', filename='checkpoint.pth.tar'):
        filepath = os.path.join(folder, filename)
        if not os.path.exists(folder):
            print("Checkpoint Directory does not exist! Making directory {}".format(folder))
            os.mkdir(folder)
        else:
            print("Checkpoint Directory exists! ")
        torch.save({
            'state_dict': self.nnet.state_dict(),
        }, filepath)

    def load_checkpoint(self, folder='checkpoint', filename='checkpoint.pth.tar'):
        # https://github.com/pytorch/examples/blob/master/imagenet/main.py#L98
        filepath = os.path.join(folder, filename)
        if not os.path.exists(filepath):
            raise ("No model in path {}".format(filepath))
        map_location = None if args.cuda else 'cpu'
        checkpoint = torch.load(filepath, map_location=map_location)
        self.nnet.load_state_dict(checkpoint['state_dict'])

In [ ]:
# Game Class

from __future__ import print_function
import sys
sys.path.append('..')
from Game import Game
import numpy as np

class HexGame(Game):
    square_content = {
        -1: "X",
        +0: "-",
        +1: "O"
    }

    @staticmethod
    def getSquarePiece(piece):
        return HexGame.square_content[piece]

    def __init__(self, n):
        self.n = n

    def getInitBoard(self):
        # return initial board (numpy board)
        b = Board(self.n)
        return np.array(b.pieces)

    def getBoardSize(self):
        # (a,b) tuple
        return (self.n, self.n)

    def getActionSize(self):
        # return number of actions
        return self.n*self.n + 1

    def getNextState(self, board, player, action):
        # if player takes action on board, return next (board,player)
        # action must be a valid move
        if action == self.n*self.n:
            return (board, -player)
        b = Board(self.n)
        b.pieces = np.copy(board)
        move = (int(action/self.n), action%self.n)
        b.execute_move(move, player)
        return (b.pieces, -player)

    def getValidMoves(self, board, player):
        # return a fixed size binary vector
        valids = [0]*self.getActionSize()
        b = Board(self.n)
        b.pieces = np.copy(board)
        legalMoves =  b.get_legal_moves(player)
        if len(legalMoves)==0:
            valids[-1]=1
            return np.array(valids)
        for x, y in legalMoves:
            valids[self.n*x+y]=1
        return np.array(valids)

    def getGameEnded(self, board, player):
        # return 0 if not ended, 1 if player 1 won, -1 if player 1 lost
        # player = 1
        b = Board(self.n)
        b.pieces = np.copy(board)
        if b.has_legal_moves(player):
            return 0
        if b.has_legal_moves(-player):
            return 0
        if b.countDiff(player) > 0:
            return 1
        return -1

    def getCanonicalForm(self, board, player):
        # return state if player==1, else return -state if player==-1
        return player*board

    def getSymmetries(self, board, pi):
        # mirror, rotational
        assert(len(pi) == self.n**2+1)  # 1 for pass
        pi_board = np.reshape(pi[:-1], (self.n, self.n))
        l = []

        for i in range(1, 5):
            for j in [True, False]:
                newB = np.rot90(board, i)
                newPi = np.rot90(pi_board, i)
                if j:
                    newB = np.fliplr(newB)
                    newPi = np.fliplr(newPi)
                l += [(newB, list(newPi.ravel()) + [pi[-1]])]
        return l

    def stringRepresentation(self, board):
        return board.tostring()

    def stringRepresentationReadable(self, board):
        board_s = "".join(self.square_content[square] for row in board for square in row)
        return board_s

    def getScore(self, board, player):
        b = Board(self.n)
        b.pieces = np.copy(board)
        return b.countDiff(player)

    @staticmethod
    def display(board):
        n = board.shape[0]
        print("   ", end="")
        for y in range(n):
            print(y, end=" ")
        print("")
        print("-----------------------")
        for y in range(n):
            print(y, "|", end="")    # print the row #
            for x in range(n):
                piece = board[y][x]    # get the piece to print
                print(OthelloGame.square_content[piece], end=" ")
            print("|")

        print("-----------------------")


In [ ]:
# Logic
'''
Author: Eric P. Nichols
Date: Feb 8, 2008.
Board class.
Board data:
  1=white, -1=black, 0=empty
  first dim is column , 2nd is row:
     pieces[1][7] is the square in column 2,
     at the opposite end of the board in row 8.
Squares are stored and manipulated as (x,y) tuples.
x is the column, y is the row.
'''
class Board():

    # list of all 8 directions on the board, as (x,y) offsets
    __directions = [(1,1),(1,0),(1,-1),(0,-1),(-1,-1),(-1,0),(-1,1),(0,1)]

    def __init__(self, n):
        "Set up initial board configuration."

        self.n = n
        # Create the empty board array.
        self.pieces = [None]*self.n
        for i in range(self.n):
            self.pieces[i] = [0]*self.n

        # Set up the initial 4 pieces.
        self.pieces[int(self.n/2)-1][int(self.n/2)] = 1
        self.pieces[int(self.n/2)][int(self.n/2)-1] = 1
        self.pieces[int(self.n/2)-1][int(self.n/2)-1] = -1;
        self.pieces[int(self.n/2)][int(self.n/2)] = -1;

    # add [][] indexer syntax to the Board
    def __getitem__(self, index): 
        return self.pieces[index]

    def countDiff(self, color):
        """Counts the # pieces of the given color
        (1 for white, -1 for black, 0 for empty spaces)"""
        count = 0
        for y in range(self.n):
            for x in range(self.n):
                if self[x][y]==color:
                    count += 1
                if self[x][y]==-color:
                    count -= 1
        return count

    def get_legal_moves(self, color):
        """Returns all the legal moves for the given color.
        (1 for white, -1 for black
        """
        moves = set()  # stores the legal moves.

        # Get all the squares with pieces of the given color.
        for y in range(self.n):
            for x in range(self.n):
                if self[x][y]==color:
                    newmoves = self.get_moves_for_square((x,y))
                    moves.update(newmoves)
        return list(moves)

    def has_legal_moves(self, color):
        for y in range(self.n):
            for x in range(self.n):
                if self[x][y]==color:
                    newmoves = self.get_moves_for_square((x,y))
                    if len(newmoves)>0:
                        return True
        return False

    def get_moves_for_square(self, square):
        """Returns all the legal moves that use the given square as a base.
        That is, if the given square is (3,4) and it contains a black piece,
        and (3,5) and (3,6) contain white pieces, and (3,7) is empty, one
        of the returned moves is (3,7) because everything from there to (3,4)
        is flipped.
        """
        (x,y) = square

        # determine the color of the piece.
        color = self[x][y]

        # skip empty source squares.
        if color==0:
            return None

        # search all possible directions.
        moves = []
        for direction in self.__directions:
            move = self._discover_move(square, direction)
            if move:
                # print(square,move,direction)
                moves.append(move)

        # return the generated move list
        return moves

    def execute_move(self, move, color):
        """Perform the given move on the board; flips pieces as necessary.
        color gives the color pf the piece to play (1=white,-1=black)
        """

        #Much like move generation, start at the new piece's square and
        #follow it on all 8 directions to look for a piece allowing flipping.

        # Add the piece to the empty square.
        # print(move)
        flips = [flip for direction in self.__directions
                      for flip in self._get_flips(move, direction, color)]
        assert len(list(flips))>0
        for x, y in flips:
            #print(self[x][y],color)
            self[x][y] = color

    def _discover_move(self, origin, direction):
        """ Returns the endpoint for a legal move, starting at the given origin,
        moving by the given increment."""
        x, y = origin
        color = self[x][y]
        flips = []

        for x, y in Board._increment_move(origin, direction, self.n):
            if self[x][y] == 0:
                if flips:
                    # print("Found", x,y)
                    return (x, y)
                else:
                    return None
            elif self[x][y] == color:
                return None
            elif self[x][y] == -color:
                # print("Flip",x,y)
                flips.append((x, y))

    def _get_flips(self, origin, direction, color):
        """ Gets the list of flips for a vertex and direction to use with the
        execute_move function """
        #initialize variables
        flips = [origin]

        for x, y in Board._increment_move(origin, direction, self.n):
            #print(x,y)
            if self[x][y] == 0:
                return []
            if self[x][y] == -color:
                flips.append((x, y))
            elif self[x][y] == color and len(flips) > 0:
                #print(flips)
                return flips

        return []

    @staticmethod
    def _increment_move(move, direction, n):
        # print(move)
        """ Generator expression for incrementing moves """
        move = list(map(sum, zip(move, direction)))
        #move = (move[0]+direction[0], move[1]+direction[1])
        while all(map(lambda x: 0 <= x < n, move)): 
        #while 0<=move[0] and move[0]<n and 0<=move[1] and move[1]<n:
            yield move
            move=list(map(sum,zip(move,direction)))
            #move = (move[0]+direction[0],move[1]+direction[1])

In [ ]:
# Main

import logging

import coloredlogs

from Coach import Coach
from othello.OthelloGame import OthelloGame as Game
from othello.pytorch.NNet import NNetWrapper as nn
from utils import *

log = logging.getLogger(__name__)

coloredlogs.install(level='INFO')  # Change this to DEBUG to see more info.

args = dotdict({
    'numIters': 1000,
    'numEps': 100,              # Number of complete self-play games to simulate during a new iteration.
    'tempThreshold': 15,        #
    'updateThreshold': 0.6,     # During arena playoff, new neural net will be accepted if threshold or more of games are won.
    'maxlenOfQueue': 200000,    # Number of game examples to train the neural networks.
    'numMCTSSims': 25,          # Number of games moves for MCTS to simulate.
    'arenaCompare': 40,         # Number of games to play during arena play to determine if new net will be accepted.
    'cpuct': 1,

    'checkpoint': './temp/',
    'load_model': False,
    'load_folder_file': ('/dev/models/8x100x50','best.pth.tar'),
    'numItersForTrainExamplesHistory': 20,

})


def main():
    log.info('Loading %s...', Game.__name__)
    g = Game(6)

    log.info('Loading %s...', nn.__name__)
    nnet = nn(g)

    if args.load_model:
        log.info('Loading checkpoint "%s/%s"...', args.load_folder_file[0], args.load_folder_file[1])
        nnet.load_checkpoint(args.load_folder_file[0], args.load_folder_file[1])
    else:
        log.warning('Not loading a checkpoint!')

    log.info('Loading the Coach...')
    c = Coach(g, nnet, args)

    if args.load_model:
        log.info("Loading 'trainExamples' from file...")
        c.loadTrainExamples()

    log.info('Starting the learning process 🎉')
    c.learn()


if __name__ == "__main__":
    main()


In [ ]:
# NN for Hex

import sys
sys.path.append('..')
from utils import *

import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class HexNNet(nn.Module):
    def __init__(self, game, args):
        # game params
        self.board_x, self.board_y = game.getBoardSize()
        self.action_size = game.getActionSize()
        self.args = args

        super(HexNNet, self).__init__()
        self.conv1 = nn.Conv2d(1, args.num_channels, 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(args.num_channels, args.num_channels, 3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(args.num_channels, args.num_channels, 3, stride=1)
        self.conv4 = nn.Conv2d(args.num_channels, args.num_channels, 3, stride=1)

        self.bn1 = nn.BatchNorm2d(args.num_channels)
        self.bn2 = nn.BatchNorm2d(args.num_channels)
        self.bn3 = nn.BatchNorm2d(args.num_channels)
        self.bn4 = nn.BatchNorm2d(args.num_channels)

        self.fc1 = nn.Linear(args.num_channels*(self.board_x-4)*(self.board_y-4), 1024)
        self.fc_bn1 = nn.BatchNorm1d(1024)

        self.fc2 = nn.Linear(1024, 512)
        self.fc_bn2 = nn.BatchNorm1d(512)

        self.fc3 = nn.Linear(512, self.action_size)

        self.fc4 = nn.Linear(512, 1)

    def forward(self, s):
        #                                                           s: batch_size x board_x x board_y
        s = s.view(-1, 1, self.board_x, self.board_y)                # batch_size x 1 x board_x x board_y
        s = F.relu(self.bn1(self.conv1(s)))                          # batch_size x num_channels x board_x x board_y
        s = F.relu(self.bn2(self.conv2(s)))                          # batch_size x num_channels x board_x x board_y
        s = F.relu(self.bn3(self.conv3(s)))                          # batch_size x num_channels x (board_x-2) x (board_y-2)
        s = F.relu(self.bn4(self.conv4(s)))                          # batch_size x num_channels x (board_x-4) x (board_y-4)
        s = s.view(-1, self.args.num_channels*(self.board_x-4)*(self.board_y-4))

        s = F.dropout(F.relu(self.fc_bn1(self.fc1(s))), p=self.args.dropout, training=self.training)  # batch_size x 1024
        s = F.dropout(F.relu(self.fc_bn2(self.fc2(s))), p=self.args.dropout, training=self.training)  # batch_size x 512

        pi = self.fc3(s)                                                                         # batch_size x action_size
        v = self.fc4(s)                                                                          # batch_size x 1

        return F.log_softmax(pi, dim=1), torch.tanh(v)


In [1]:
class hexPosition (object):
    """
    The class hexPosition stores data on a hex board position. The slots of an object are: size (an integer between 2 and 26), board (an array, 0=noStone, 1=whiteStone, 2=blackStone), and winner (0=noWin, 1=whiteWin, 2=blackWin).
    """
   
    def __init__ (self, size=5):
        
        if size > 9:
            print("Warning: Large board size, position evaluation may be slow.")
        
        self.size = max(2,min(size,26))
               
        self.board = [[0 for x in range(max(2,min(size,26)))] for y in range(max(2,min(size,26)))]
        self.winner = 0
                
    def reset (self):
        """
        This method resets the hex board. All stones are removed from the board.
        """
        
        self.board = [[0 for x in range(self.size)] for y in range(self.size)]
        self.winner = 0
        
    def printBoard (self, invert_colors=True):
        """
        This method prints a visualization of the hex board to the standard output. If the standard output prints black text on a white background, one must set invert_colors=False.
        """
        
        names = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        indent = 0
        headings = " "*5+(" "*3).join(names[:self.size])
        print(headings)
        tops = " "*5+(" "*3).join("_"*self.size)
        print(tops)
        roof = " "*4+"/ \\"+"_/ \\"*(self.size-1)
        print(roof)
        
        #Attention: Color mapping inverted by default for display in terminal.
        if invert_colors:
            color_mapping = lambda i: " " if i==0 else ("\u25CB" if i==2 else "\u25CF")
        else:
            color_mapping = lambda i: " " if i==0 else ("\u25CF" if i==2 else "\u25CB")
        
        for r in range(self.size):
            row_mid = " "*indent
            row_mid += "   | "
            row_mid += " | ".join(map(color_mapping,self.board[r]))
            row_mid += " | {} ".format(r+1)
            print(row_mid)
            row_bottom = " "*indent
            row_bottom += " "*3+" \\_/"*self.size
            if r<self.size-1:
                row_bottom += " \\"
            print(row_bottom)
            indent += 2
        headings = " "*(indent-2)+headings
        print(headings)

    def getAdjacent (self, position):
        """
        Helper function to obtain adjacent cells in the board array.
        """
        
        u = (position[0]-1, position[1])
        d = (position[0]+1, position[1])
        r = (position[0], position[1]-1)
        l = (position[0], position[1]+1)
        ur = (position[0]-1, position[1]+1)
        dl = (position[0]+1, position[1]-1)
        
        return [pair for pair in [u,d,r,l,ur,dl] if max(pair[0], pair[1]) <= self.size-1 and min(pair[0], pair[1]) >= 0]

    def getActionSpace (self, recodeBlackAsWhite=False):
        """
        This method returns a list of array positions which are empty (on which stones may be put).
        """
        
        actions = []
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == 0:
                    actions.append((i,j))
        if recodeBlackAsWhite:
            return [self.recodeCoordinates(action) for action in actions]
        else:
            return(actions)
    
    def playRandom (self, player):
        """
        This method returns a uniformly randomized valid moove for the chosen player (player=1, or player=2).
        """
        from random import choice
        
        chosen = choice(self.getActionSpace())
        self.board[chosen[0]][chosen[1]] = player
        
    def randomMatch (self, evaluate_when_full=False):
        """
        This method randomizes an entire playthrough. Mostly useful to test code functionality. If evaluate_when_full=True then the board will be completely filled before the position is evaluated. Otherwise evaluation happens after every moove.
        """
        
        player = 1
        
        if evaluate_when_full:
            for i in range(self.size**2):
                self.playRandom(player)
                if (player==1):
                    self.whiteWin()
                    player = 2
                else:
                    self.blackWin()
                    player = 1
            self.whiteWin()
            self.blackWin()
        
        while self.winner == 0:
            self.playRandom(player)
            if (player==1):
                self.whiteWin()
                player = 2
            else:
                self.blackWin()
                player = 1
                
    def prolongPath (self, path):
        """
        A helper function used for board evaluation.
        """
        
        from copy import deepcopy
        
        player = self.board[path[-1][0]][path[-1][1]]
        candidates = self.getAdjacent(path[-1])
        candidates = list(filter(lambda cand: cand not in path, candidates))
        candidates = list(filter(lambda cand: self.board[cand[0]][cand[1]] == player, candidates))
                
        return [deepcopy(path)+[cand] for cand in candidates]
    
    def whiteWin (self, verbose=False):
        """
        Evaluate whether the board position is a win for 'white'. Uses breadth first search. If verbose=True a winning path will be printed to the standard output (if one exists). This method may be time-consuming, especially for larger board sizes.
        """
        
        paths = []
        for i in range(self.size):
            if self.board[i][0] == 1:
                paths.append([(i,0)])
                
        while True:
            
            if len(paths) == 0:
                return False
                                    
            for path in paths:
                
                prolongations = self.prolongPath(path)
                paths.remove(path)
                
                for new in prolongations:
                    if new[-1][1] == self.size-1:
                        if verbose:
                            print("A winning path for White:\n",new)
                        self.winner = 1
                        return True
                    paths.append(new)
    
    def blackWin (self, verbose=False):
        """
        Evaluate whether the board position is a win for 'black'. Uses breadth first search. If verbose=True a winning path will be printed to the standard output (if one exists). This method may be time-consuming, especially for larger board sizes.
        """
        
        paths = []
        for i in range(self.size):
            if self.board[0][i] == 2:
                paths.append([(0,i)])
                
        while True:
            
            if len(paths) == 0:
                return False
                                    
            for path in paths:
                
                prolongations = self.prolongPath(path)
                paths.remove(path)
                
                for new in prolongations:
                    if new[-1][0] == self.size-1:
                        if verbose:
                            print("A winning path for Black:\n",new)
                        self.winner = 2
                        return True
                    paths.append(new)
    
    def humanVersusMachine (self, human_player=1, machine=None):
        """
        Play a game against an AI. The variable machine must point to a function that maps a state vector and an action set to an element of the action set. If machine is not specified random actions will be used.
        """
        
        if machine == None:
            def machine (state_list, action_list):
                from random import choice 
                return choice(action_list)
        
        self.reset()
        
        def translator (string):
            #This function translates human terminal input into the proper array indices.
            
            number_translated = 27
            letter_translated = 27
            
            names = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
            
            if len(string) > 0:
                letter = string[0]
            if len(string) > 1:
                number1 = string[1]
            if len(string) > 2:
                number2 = string[2]
            
            for i in range(26):
                if names[i] == letter:
                    letter_translated = i
                    break
            
            if len(string) > 2:
                for i in range(10,27):
                    if number1 + number2 == "{}".format(i):
                        number_translated = i-1
            else:
                for i in range(1,10):
                    if number1 == "{}".format(i):
                        number_translated = i-1
                    
            return (number_translated, letter_translated)
            
        while self.winner == 0:
            
            self.printBoard()
            
            possible_actions = self.getActionSpace()
            
            human_input = (27,27)
            
            while human_input not in possible_actions:
                
                human_input = translator(input("Enter your moove (e.g. 'A1'): "))
                
            self.board[human_input[0]][human_input[1]] = 1
            
            self.whiteWin()
                            
            if self.winner == 1:
                self.printBoard()
                print ("The human player (White) has won!")
                self.whiteWin(verbose=True)
            else:
                blacks_moove = machine(self.getStateVector(), self.getActionSpace())
                self.board[blacks_moove[0]][blacks_moove[1]] = 2
                
                self.blackWin()
                if self.winner == 2:
                    self.printBoard()
                    print("The computer (Black) has won!")
                    self.blackWin(verbose=True)
                    
    def recodeBlackAsWhite (self, printBoard=False, invert_colors=True):
        """
        Returns a board where black is recoded as white and wants to connect horizontally. This corresponds to flipping the board long the south-west to north-east diagonal.
        """
        flipped_board = [[0 for i in range(self.size)] for i in range(self.size)]
              
        #flipping and color change
        for i in range(self.size):
            for j in range(self.size):
                if self.board[self.size-1-j][self.size-1-i] == 1:
                    flipped_board[i][j] = 2
                if self.board[self.size-1-j][self.size-1-i] == 2:
                    flipped_board[i][j] = 1
        #return flipped_board
    
        names = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        indent = 0
        headings = " "*5+(" "*3).join(names[:self.size])
        print(headings)
        tops = " "*5+(" "*3).join("_"*self.size)
        print(tops)
        roof = " "*4+"/ \\"+"_/ \\"*(self.size-1)
        print(roof)
        
        #Attention: Color mapping inverted by default for display in terminal.
        if invert_colors:
            color_mapping = lambda i: " " if i==0 else ("\u25CB" if i==2 else "\u25CF")
        else:
            color_mapping = lambda i: " " if i==0 else ("\u25CF" if i==2 else "\u25CB")
        
        for r in range(self.size):
            row_mid = " "*indent
            row_mid += "   | "
            row_mid += " | ".join(map(color_mapping,flipped_board[r]))
            row_mid += " | {} ".format(r+1)
            print(row_mid)
            row_bottom = " "*indent
            row_bottom += " "*3+" \\_/"*self.size
            if r<self.size-1:
                row_bottom += " \\"
            print(row_bottom)
            indent += 2
        headings = " "*(indent-2)+headings
        print(headings)
        
    def recodeCoordinates (self, coordinates):
        """
        Transforms a coordinate tuple (with respect to the board) analogously to the method recodeBlackAsWhite.
        """
        return (self.size-1-coordinates[1], self.size-1-coordinates[0])
    
    def coordinate2scalar (self, coordinates):
        """
        Helper function to convert coordinates to scalars.
        """
        return coordinates[0]*self.size + coordinates[1]
    
    def scalar2coordinates (self, scalar):
        """
        Helper function to transform a scalar "back" to coordinates.
        """
        temp = int(scalar/self.size)
        return (temp, scalar-temp*self.size)


# #Initializing an object
# myboard = hexPosition(size=4)
# #Display the board in standard output
# myboard.printBoard()
# #Random playthrough
# myboard.randomMatch(evaluate_when_full=False)
# myboard.printBoard()
# myboard.blackWin(verbose=True)
# myboard.whiteWin(verbose=True)

# myboard.recodeBlackAsWhite(printBoard=True)

# #check whether Black has won
# myboard.blackWin(verbose=True)
# #check whether White has won
# myboard.whiteWin(verbose=True)
# #print board with inverted colors
# myboard.getInvertedBoard()
# #get board as vector
# myboard.getStateVector(inverted=False)
# #reset the board
# myboard.reset()
# #play against random player
# myboard.humanVersusMachine()



In [17]:

b = hexPosition(size=4)

player = 1

while(1):
    b.playRandom(player)
    
    if(b.whiteWin(True)):
        break
    if(b.blackWin(True)):
        break       
    if(player == 1):
        player = 2
    else:
        player = 1



A winning path for Black:
 [(0, 2), (1, 1), (1, 2), (2, 2), (3, 1)]
